In [ ]:
import numpy as np
from tensorflow import keras
from keras import models
from keras import layers
from keras import backend as K
from keras.layers.core import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import activations
import tensorflow as tf

# Loading in inputs and outputs

In [ ]:
load=np.load('shuffled_inputs.npz')

In [ ]:
Inputs=load['arr_0']

In [ ]:
load=np.load('shuffled_outputs.npz')

In [ ]:
Outputs=load['arr_0']

# Seperating into training, validation and testing sets and normalising each feature

In [ ]:
testing_inputs=Inputs[918000:,:]
testing_outputs=Outputs[918000:,:]
validation_inputs=Inputs[756000:918000,:]
validation_outputs=Outputs[756000:918000,:]
training_inputs=Inputs[:756000,:]
training_outputs=Outputs[:756000,:]

In [ ]:
means=training_inputs.mean(axis=0)
std=training_inputs.std(axis=0)
training_inputs-=means
training_inputs/=std
testing_inputs-=means
testing_inputs/=std
validation_inputs-=means
validation_inputs/=std

# Creating custom activation functions

Msigmoid

In [ ]:
def Msigmoid(x):
  new=keras.activations.sigmoid(x)
  threshold=0.00001
  new=tf.where(tf.greater(new,threshold),new,0.0)
  return(new)

get_custom_objects().update({'Msigmoid': Activation(Msigmoid)})

MReLU

In [ ]:
def MReLU(x):
  new=keras.activations.relu(x)
  return(K.minimum(new,1))

get_custom_objects().update({'MReLU': Activation(MReLU)})

# Restricting vertical levels

In [ ]:
temporary1=training_inputs[:,0:37]
temporary1=np.append(temporary1,training_inputs[:,70:107],axis=1)
temporary1=np.append(temporary1,training_inputs[:,140:177],axis=1)
temporary1=np.append(temporary1,training_inputs[:,210:214],axis=1)
training_inputs=temporary1
training_outputs=training_outputs[:,0:37]

# Different samples for training
In each of the subheadings below the sample for training is created and then shuffled as the way the samples are created it puts similar values together. Only run samples individually as to prevent using too much RAM and the notebook crashing, each sample rewrites the array rather than creating new ones.

Sample 1

In [ ]:
for i in range(8,30):
  indx=np.where(np.logical_and((training_outputs).sum(axis=1)>i,(training_outputs).sum(axis=1)<=i+1))
  num_repeats=int(np.floor(80000/np.shape(indx)[1])) 
  training_outputs=np.append(training_outputs,np.repeat(training_outputs[indx],num_repeats,axis=0),axis=0) 
  training_inputs=np.append(training_inputs,np.repeat(training_inputs[indx],num_repeats,axis=0),axis=0)

In [ ]:
load=np.load('permutation2.npz')

In [ ]:
perm=load['arr_0']

In [ ]:
for i in range(0,214):
  training_inputs[:,i]=training_inputs[:,i][perm]
for i in range(0,70):
  training_outputs[:,i]=training_outputs[:,i][perm]

Sample 2

In [ ]:
load=np.load('perm_to_decide_on_the_quater.npz')

In [ ]:
perm=load['arr_0']

In [ ]:
indx=np.where((training_outputs).sum(axis=1)<1)
not_indx=np.where((training_outputs).sum(axis=1)>=1)
others_out=training_outputs[not_indx]
others_inp=training_inputs[not_indx]
subset_out=training_outputs[indx]
subset_inp=training_inputs[indx]
shuffled_subset_out=subset_out[perm]
shuffled_subset_inp=subset_inp[perm]
del subset_out
del subset_inp
training_outputs=np.append(others_out,shuffled_subset_out[:int(np.floor(shuffled_subset_out.shape[0]/4))],axis=0)
training_inputs=np.append(others_inp,shuffled_subset_inp[:int(np.floor(shuffled_subset_out.shape[0]/4))],axis=0)
del shuffled_subset_out
del shuffled_subset_inp

In [ ]:
load=np.load('final_perm_for_data_once_in_quater_subset.npz')

In [ ]:
perm=load['arr_0']

In [ ]:
for i in range(0,214):
  training_inputs[:,i]=training_inputs[:,i][perm]
for i in range(0,70):
  training_outputs[:,i]=training_outputs[:,i][perm]

Sample 3

In [ ]:
indx=np.where((training_outputs).sum(axis=1)>1)
training_inputs_new=np.append(training_inputs,training_inputs[indx],axis=0)
training_inputs_new=np.append(training_inputs_new,training_inputs[indx],axis=0)
training_inputs=np.append(training_inputs_new,training_inputs[indx],axis=0)
del training_inputs_new
training_outputs_new=np.append(training_outputs,training_outputs[indx],axis=0)
training_outputs_new=np.append(training_outputs_new,training_outputs[indx],axis=0)
training_outputs=np.append(training_outputs_new,training_outputs[indx],axis=0)
del training_outputs_new

In [ ]:
load=np.load('permutation4.npz')

In [ ]:
perm=load['arr_0']

In [ ]:
for i in range(0,214):
  training_inputs[:,i]=training_inputs[:,i][perm]
for i in range(0,70):
  training_outputs[:,i]=training_outputs[:,i][perm]

# Building the neural network

The code below can be adjusted for all the cases in the project.

*   For data with the whole 70 levels the input_dim needs to be 214 and the number of neurons in the final model.add(layer.Dense needs to be 70.
*   For data with 37 levels the input_dim needs to be 115 and the number of neurons in the final model.add(layer.Dense needs to be 37.
*   For sigmoid as the final layer actiation, the final model.add(layer.Dense needs to have activation='sigmoid' and all other model.add(layers.Dense need to have activation='relu'
*   For Msigmoid as the final layer actiation, the final model.add(layer.Dense needs to have activation='Msigmoid' and all other model.add(layers.Dense need to have activation='relu'
*   For MReLU as the final layer actiation, the final model.add(layer.Dense needs to have activation='MReLU' and all other model.add(layers.Dense need to have activation='LeakyReLU'





In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
def build_model():
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_dim=214))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(70, activation='sigmoid'))
  model.compile(optimizer=opt, loss='mse', metrics=['mae'])
  return model

# Training and predictions

The code below creates the neural network and runs training.

In [ ]:
model=build_model()
history=model.fit(training_inputs,training_outputs,epochs=100,batch_size=64,validation_data=(validation_inputs,validation_outputs),verbose=1)

The code below uses the network to predicted the cloud fraction in the test set.

In [ ]:
predicted_values=model.predict(testing_inputs, batch_size=3000,verbose=2)

Below saves the loss and mae for the validation and raining sets.

In [ ]:
val_mae=history.history["val_mae"]
training_mae=history.history["mae"]
val_loss=history.history["val_loss"]
training_loss=history.history["loss"]